# Evaluation of RL Algos

In [1]:
import os
import pandas as pd
import numpy as np

from utils import backtestWs, weightsEw
from utils import test_static_agent
from utils import create_weights_gif

from metrics import summary_stats
from satis_models import sharpe_ratio
from pandas.tseries.offsets import BDay

from agents import Static
from environment import StaticEnvironment
from mom_models import sample_cov, mean_return_historic, ema_return_historic, elton_gruber_cov

## Notebook Parameters 

In [2]:
#paths
RESULTS_PATH = os.path.join('C:\\', 'Users','matus','gdrive','RL_RESULTS')
DATA_PATH = os.path.join('C:\\', 'Users','matus','gdrive','datasets', 'Research')


#model parameters
MODELS = ['A2C', 'DDPG', 'PPO','SAC','TD3']

RISK_FREE = '00_db_10YTBILL__PX_LAST.csv'
FACTORS = '00_db_FACTORS__PX_LAST.xlsx'

REBALANCING_WINDOW = {'Weekly': 5,
                      'Monthly': 21,
                      'Quarterly': 63,
                      'Semi-annually': 128,
                     }
#                       'Annually': 252}
DATASET = '00_db_BE500_MOMSIZE100'

# Load results and Data

In [3]:
weights_RL = {mod : pd.read_csv(os.path.join(RESULTS_PATH,DATASET,mod,str.lower(mod) + '_actions.csv'),index_col=0,parse_dates=True) for mod in MODELS}
r_full = pd.read_csv(os.path.join(DATA_PATH, DATASET + '__PX_LAST.csv'),index_col=0,parse_dates=True)
test_index = weights_RL['A2C'].index

r = r_full.loc[test_index, weights_RL['A2C'].columns]
rf = pd.read_csv(os.path.join(DATA_PATH, RISK_FREE), index_col = 0, parse_dates = True).loc[test_index]
rf /= 100 #is in percentages
rf = rf.iloc[:,0] #take the 10Y US T-Bill

db_factors = pd.read_excel(os.path.join(DATA_PATH, FACTORS), index_col = 0, parse_dates = True)

# Trade Test

## Reinforcement Learning Algorithms 

In [4]:
backtests = dict()
backtests_cuml = dict()
for window in REBALANCING_WINDOW:
    backtests[window] = {mod : backtestWs(r,weights_RL[mod], estimation_window = 0, 
                                          rebalancing_window = REBALANCING_WINDOW[window]) for mod in MODELS}
    backtests[window]['EW'] = backtestWs(r,weightsEw,0,REBALANCING_WINDOW[window])
    
    backtests_cuml[window] = (1+pd.DataFrame(backtests[window])).cumprod()
    backtests_cuml[window].iloc[0,:] = 1

## Static Quadratic Rebalance 

In [5]:
env_kwargs = {
    'equity': 1_000_000,
    'transaction_cost': 0.001,
    'mean_model': mean_return_historic,
    'cov_model': sample_cov,
    'satisfaction_model': sharpe_ratio,
}

In [6]:
fitting_period = 2*252
start_date = r.index[0]
end_date = r.index[-1]

start_fit = (pd.to_datetime(start_date) - BDay(fitting_period)).strftime('%Y-%m-%d')

env = StaticEnvironment(r_full.loc[start_fit : end_date, weights_RL['A2C'].columns] ,**env_kwargs)
agent = Static(mean_return_historic, sample_cov, len(r.columns), allow_short=False)

agent.set_factor_bound(0)
agent.set_weight_bound((0.01,1))

In [7]:
for window in REBALANCING_WINDOW:
    #minimum-variance portfolio
    try: 
        mv = test_static_agent(env, agent, 'minimum_variance', fitting_period, REBALANCING_WINDOW[window])
        backtests[window]['MV'] =  mv[:len(r)]
        del mv
    except:
        print('QSQP Solver failed, or no feasible solution can be reached.')
    
    #maximum-sharpe portfolio
    try:
        ms = test_static_agent(env, agent, 'max_sharpe', fitting_period, REBALANCING_WINDOW[window])
        backtests[window]['MSR'] =  ms[:len(r)]
        del ms
        
    except:
        print('QSQP Solver failed, or no feasible solution can be reached.')
    
    #factor-constrained portfolio 0.2
    try:
        fs = test_static_agent(env, agent, 'sr_factor', fitting_period, REBALANCING_WINDOW[window], factors = db_factors, **env_kwargs)
        backtests[window]['FC'] =  fs[:len(r)]
        del fs
    except:
        print('QSQP Solver failed, or no feasible solution can be reached.')
        
    try:
        er = test_static_agent(env, agent, 'equal_risk', fitting_period, REBALANCING_WINDOW[window])
        backtests[window]['ER'] = er[:len(r)]
        del er
    except:
        print('QSQP Solver failed, or no feasible solution can be reached.')
    

Tested minimum_variance in 2.5690 seconds
Tested max_sharpe in 442.6072 seconds
Tested sr_factor in 831.6468 seconds
Tested equal_risk in 59.8491 seconds
Tested minimum_variance in 0.6220 seconds
Tested max_sharpe in 104.4617 seconds
Tested sr_factor in 198.2623 seconds
Tested equal_risk in 13.7973 seconds
Tested minimum_variance in 0.2220 seconds
Tested max_sharpe in 37.6155 seconds
Tested sr_factor in 70.3496 seconds
Tested equal_risk in 4.6961 seconds
Tested minimum_variance in 0.1406 seconds
Tested max_sharpe in 21.6277 seconds
Tested sr_factor in 39.7067 seconds
Tested equal_risk in 2.4554 seconds


# Save results 

In [8]:
for key in backtests.keys():
    df = pd.DataFrame(backtests[str(key)])
    df.to_csv(os.path.join(RESULTS_PATH, DATASET, str(key) + '.csv'))

# Create Weights GIF

In [9]:
for mod in MODELS:
    print(f'Creating GIF for {mod}')
    
    if not os.path.exists(os.path.join(RESULTS_PATH, DATASET, mod, 'gif')):
        os.makedirs(os.path.join(RESULTS_PATH, DATASET, mod, 'gif'))
        
    create_weights_gif(weights_RL[mod], mod, os.path.join(RESULTS_PATH, DATASET, mod))

Creating GIF for A2C
27it [03:27,  7.69s/it]                                                                                                
Charts saved 

Creating gif

Gif produced in 3.6165 minutes
DONE
Creating GIF for DDPG
27it [03:33,  7.91s/it]                                                                                                
Charts saved 

Creating gif

Gif produced in 3.7134 minutes
DONE
Creating GIF for PPO
27it [03:24,  7.57s/it]                                                                                                
Charts saved 

Creating gif

Gif produced in 3.5860 minutes
DONE
Creating GIF for SAC
27it [03:31,  7.82s/it]                                                                                                
Charts saved 

Creating gif

Gif produced in 3.6965 minutes
DONE
Creating GIF for TD3
27it [03:40,  8.16s/it]                                                                                                
Charts saved 

Creating gif

Gif p